In [24]:
import numpy as np 
import pandas as pd
from IPython.display import display

In [8]:
raw_rental_data = pd.read_csv(r'C:\Users\herms\OneDrive\Documentos\AidaDocuments\York Online Computer Science Masters\BDA\SummativeAssignment\CleaningAndPreprocessingPython\housing_train.csv' )

In [9]:
raw_rental_data.head()

,id,url,region,region_url,rent,type,sqfeet,bedrooms,bathrooms,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,demand,description,latitude,long,state
0,7039061606,https://bham.craigslist.org/apa/d/birmingham-h...,birmingham,https://bham.craigslist.org,1195,apartment,1908,3,2.0,1,...,0,0,0,laundry on site,street parking,yes,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
1,7041970863,https://bham.craigslist.org/apa/d/birmingham-w...,birmingham,https://bham.craigslist.org,1120,apartment,1319,3,2.0,1,...,0,0,0,laundry on site,off-street parking,yes,Find Your Way to Haven Apartment Homes Come ho...,33.3755,-86.8045,al
2,7041966914,https://bham.craigslist.org/apa/d/birmingham-g...,birmingham,https://bham.craigslist.org,825,apartment,1133,1,1.5,1,...,0,0,0,laundry on site,street parking,yes,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
3,7041966936,https://bham.craigslist.org/apa/d/birmingham-f...,birmingham,https://bham.craigslist.org,800,apartment,927,1,1.0,1,...,0,0,0,laundry on site,street parking,yes,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al
4,7041966888,https://bham.craigslist.org/apa/d/birmingham-2...,birmingham,https://bham.craigslist.org,785,apartment,1047,2,1.0,1,...,0,0,0,laundry on site,street parking,yes,Apartments In Birmingham AL Welcome to 100 Inv...,33.4226,-86.7065,al


In [10]:
#rental_data_ver1 does not have columns: 'url', 'region_url' and 'description' 
rental_data_ver1 = raw_rental_data.drop(columns=['url', 'region_url', 'description'])

In [11]:
for column in rental_data_ver1.columns:
    print(f'\n************')
    print(f'\nValues by attribute {column}: ')
    print(rental_data_ver1[column].value_counts())
    print(f'\nValues in {column} that are NaN:')
    print(rental_data_ver1[column].isna().sum())


************

Values by attribute id: 
7039061606    1
7041048843    1
7049243882    1
7046886383    1
7049243263    1
             ..
7045108982    1
7045102272    1
7049414444    1
7041016901    1
7048975938    1
Name: id, Length: 36309, dtype: int64

Values in id that are NaN:
0

************

Values by attribute region: 
inland empire           2561
tucson                  2465
los angeles             2260
fresno / madera         2195
huntsville / decatur    2164
                        ... 
texoma                     1
susanville                 1
east oregon                1
colorado springs           1
stillwater                 1
Name: region, Length: 83, dtype: int64

Values in region that are NaN:
0

************

Values by attribute rent: 
850     429
995     403
1200    378
950     358
1250    335
       ... 
921       1
5895      1
4234      1
936       1
982       1
Name: rent, Length: 2027, dtype: int64

Values in rent that are NaN:
0

************

Values by attribute 

In [18]:
demand_values = rental_data_ver1['demand'].value_counts()


In [27]:
display(demand_values)

yes                                                            34678
no                                                               938
https://images.craigslist.org/00C0C_gY5moMhKQOH_600x450.jpg       31
https://images.craigslist.org/00X0X_459Esu0yVx5_600x450.jpg       13
https://images.craigslist.org/00Y0Y_ethkrBm4oRb_600x450.jpg       13
https://images.craigslist.org/00808_5iGPuaJtYpb_600x450.jpg       11
https://images.craigslist.org/00z0z_gzkFphVGItP_600x450.jpg       10
https://images.craigslist.org/00J0J_hOfXN7HyReX_600x450.jpg        9
https://images.craigslist.org/00p0p_1ADgASthIyz_600x450.jpg        8
https://images.craigslist.org/01515_aqyV8nVl7yk_600x450.jpg        8
https://images.craigslist.org/00N0N_3tVqEmAUNXn_600x450.jpg        8
https://images.craigslist.org/00H0H_hTc6o8Qnh0M_600x450.jpg        8
https://images.craigslist.org/00D0D_k5svQ6NuOXu_600x450.jpg        7
https://images.craigslist.org/00P0P_hRYVFmpxUtM_600x450.jpg        7
https://images.craigslist.org/00E0